In [1]:
import sys
import os
import glob
import re
import xarray as xr
import numpy as np
import pandas as pd
import gc
import cartopy.crs as ccrs

sys.path.append('modules')
from read_deterministic_data import read_gfs_deterministic
import calc_funcs as cfuncs
from plotter import plot_fields

F=6
fdate='2025092600'

print('Reading {0} ...'.format(F))
#########################
### READ NEW GFS DATA ###
#########################
path_to_data = '/data/projects/external_datasets/GFS/processed/*/'
if fdate is None:
    list_of_files = glob.glob(path_to_data)
    fpath = max(list_of_files, key=os.path.getctime)
    regex = re.compile(r'\d+')
    date_string = regex.findall(fpath)[-1]
elif fdate is not None:
    date_string = fdate
    fpath = '/data/projects/external_datasets/GFS/processed/{0}/'.format(date_string)

fname = '{0}_F{1}.grb2'.format(date_string, str(F).zfill(3))

fname = fpath+fname

gfs_vardict = {
               
                "rwmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'rwmr'}, # rain mixing ratio (kg/kg)
                "clwmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'clwmr'}, # cloud water mixing ratio (kg/kg)
                "icmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'icmr'}, # ice mixing ratio (kg/kg)
                "snmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'snmr'},  # snow mixing ratio (kg/kg)
                "u":{'typeOfLevel': 'isobaricInhPa', "shortName":'u'}, #U-component of wind
                "v":{'typeOfLevel': 'isobaricInhPa', "shortName":'v'}, #V-component of wind
                "t":{'typeOfLevel': 'isobaricInhPa', "shortName":'t'}, #Temperature
                "q":{'typeOfLevel': 'isobaricInhPa', "shortName":'q'}, #Specific humidity
                "sp": {'typeOfLevel': 'surface',  'level': 0, 'paramId': 134, 'shortName': 'sp'} # surface pressure
                }

gfs = read_gfs_deterministic(fname, gfs_vardict, show_catalog=False)

# --- merge all selected vars to single dataset ---
ds = xr.merge(gfs.values())

# --- add condensates together as new var ---
ds['twmr'] = ds['q'] + ds['rwmr'] + ds['clwmr'] + ds['icmr'] + ds['snmr']

# --- Compute IVT and IWT ---
ivt_ds = cfuncs.calc_transport_manual(ds, include_condensates=False)
iwt_ds = cfuncs.calc_transport_manual(ds, include_condensates=True)
ds = xr.merge([ivt_ds, iwt_ds])

# --- trash collection ---
gc.collect()

# --- Compute ICT ---
ds['ict'] = ds['iwt'] - ds['ivt']

# --- Compute ICT/IVT ratio expressed as a percent (%) ---
ds['ratio'] = (ds['ict']/ds['ivt']) * 100

# --- Add attributes necessary for plotting function ---
new_attrs = {
        "model": "GFS",
        "init": ds.time.values,
        "valid_time": ds.valid_time.values,
        "datacrs": ccrs.PlateCarree(central_longitude=0),
    }
ds.attrs = new_attrs

# Plot - use pconfig to change vars plotted
# outpath='/data/projects/operations/Deterministic_Maps/outputs/gfs_deterministic/'
outpath='figs/'
pconfig = {
    'IVT': {
        'cfkey': 'ivt',
        'ckey': None,
        'ukey': 'ivtu',
        'vkey': 'ivtv',
        'title': "NCEP GFS IVT (kg m-1 s-1; shaded and vectors)"
    },
    'IWT': {
        'cfkey': 'ivt',
        'ckey': None,
        'ukey': 'ivtu',
        'vkey': 'ivtv',
        'title': "NCEP GFS IWT (kg m-1 s-1; shaded and vectors)"
    },
    'ICT': {
        'cfkey': 'ict',
        'ckey': None,
        'ukey': None,
        'vkey': None,
        'title': "NCEP GFS ICT (kg m-1 s-1; shaded)"
    },
    'RATIO': {
        'cfkey': 'ratio',
        'ckey': None,
        'ukey': None,
        'vkey': None,
        'title': "NCEP GFS ICT/IVT Ratio (-; shaded)"
    }
}


for varname in ['IVT', 'IWT', 'ICT', 'RATIO']:
    print(f'Plotting {varname}')
  
    config = {
            "model_name":"GFS",
            "output_fname" : outpath+'{model_name}_'+pconfig[varname]['cfkey']+'_{domain_name}_latest_F{lead_time}.png',
            "filled_var" : pconfig[varname]['cfkey'],
            "contour_var" : pconfig[varname]['ckey'],
            "vector_vars" : [pconfig[varname]['ukey'],pconfig[varname]['vkey']],
            "domain_dash_line" : False,
            "required_domains" :  ['intwest'],
            "ulc_title":pconfig[varname]['title'],
            "llc_title":"Initialized: {init_time}",
            "lrc_title":"F-{lead_time:03d}  Valid: {valid_time}",
            "logo_file":'/home/dnash/repos/integrated_cloud_water_transport_calculation/CW3E-Logo-Vertical-Acronym-FullColor.png',
            "font_file":'/home/dnash/repos/integrated_cloud_water_transport_calculation/modules/helvetica.ttc'
        }


    plot_fields(ds, config)

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed


Reading 6 ...


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'gini' loading failed:
Struct() takes at most 1 argument (3 given)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


Plotting IVT


TypeError: Input z must be at least a (2, 2) shaped array, but has shape (121, 0)